<a href="https://colab.research.google.com/github/abhilash6073/google_colab/blob/main/Agent_with_two_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-community langchain-openai langchain-core langchain-chroma
!pip install faiss-cpu
!pip install beautifulsoup4
!pip install gradio-client==0.2.10
!pip install gradio==3.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.8 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-3.5-turbo", temperature=0)
answer = llm.invoke("What is the capital of India?")
print(answer.content)

The capital of India is New Delhi.


Downloading data from a web url

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Apple_Inc")
doc = loader.load()


In [4]:
# embeddings

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"])

In [13]:
#text splitter

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(doc)

In [14]:
# vector store

from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(documents, embedding)


In [15]:
# creating the chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following based ONLY on the provided context
    <context>
    {context}
    </context>
    Question: {input}

    """
)

document_chain= create_stuff_documents_chain(llm, prompt)

In [23]:
# creating the retriever
from langchain_classic.chains.retrieval import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [29]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output = retrieval_chain.invoke({"input":"When was apple founded?"})
print(output['answer'])


Apple was founded on April 1, 1976.


In [30]:
# defining the retriever as a tool
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "Apple_Search", "Use this tool to search for any information about the company Apple")

In [31]:
#creating a tool for web search using tavily

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
from langchain_community.tools.tavily_search import TavilySearchResults
tavilty_search = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])

/tmp/ipython-input-2032498648.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavilty_search = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])


In [32]:
# cretaing a list of the available tools
tools = [retriever_tool, tavilty_search]

In [39]:
#creating an agent and assigning prompt

from langchain_classic import hub
from langchain_classic.agents import create_openai_functions_agent
from langchain_classic.agents import AgentExecutor

# importing prompt template from the hub
prompt_template = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt=prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)


In [40]:
result = agent_executor.invoke({"input":"Who founded Apple?"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `Apple_Search` with `{'query': 'founder of Apple'}`


History
Main article: History of Apple Inc.
1976–1980: Founding and incorporation
In 1976, Steve Jobs and Steve Wozniak co-founded Apple in Jobs's parents' home on Crist Drive in Los Altos, California.[6] Wozniak called the popular belief that the company was founded in the garage "a bit of a myth",[7] although they moved some operations to the garage when the bedroom became too crowded.[8]
The Apple I is Apple's first product, designed by Wozniak and sold as an assembled circuit board without the required keyboard, monitor, power supply, and the optional case.

Apple Computer, Inc. was incorporated in Cupertino, California,[2] on January 3, 1977,[17][18] without Wayne, who had left and sold his share of the company back to Jobs and Wozniak for $800 only twelve days after having co-founded it.[19] Multimillionaire Mike Markkula provided essential business expertise and funding of $2

In [41]:
# creating the chat ui using gradio

import gradio as gr


In [42]:
# defining a function
def predict(message, _):
  result = agent_executor.invoke({"input":message})
  return result['output']

In [43]:
# designing the UI
gr.ChatInterface(predict,
                 chatbot = gr.Chatbot(height=300),
                 textbox = gr.Textbox(placeholder="Hi, I'm your virtual assistant. How can I help you today?", container=False, scale=7),
                 title = "Document QA Bot",
                 theme = "soft",
                 examples = ["What happened after the launch of Apple?", "What is the weather like in Bangalore?"],
                 retry_btn=None,
                 undo_btn="Delete Previous",
                 clear_btn="Clear").launch(share=True)

IMPORTANT: You are using gradio version 3.38.0, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5e05cc751512636ae4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
